In [1]:
from inference import FastGM
from inference import FastFactor
from sampling import SampleGenerator
from inference import populate_gradient_factors
from data import DataLoader
from data import create_data_loaders
from data import DataPreprocessor
from neural_networks import Trainer
from neural_networks.losses import *
import numpy as np
import torch
import torchinfo
import torch.nn as nn
from neural_networks import Net

In [2]:

ib = 5
Z = 303.0859680175781
# uai_file = "/home/cohenn1/SDBE/width_under_20_problems/grid10x10.f10.uai"
uai_file = "/home/cohenn1/SDBE/width20-30/grid20x20.f2.uai"
# problem_name = "grid10x10.f10-ib" + str(ib)
problem_name = "grid20x20.f2-ib" + str(ib)
output_path = "/home/cohenn1/SDBE/PyTorchGMs/graphs"
grid20f2_idxs = [30, 106, 213, 123, 331]
# idx = 74 # for grid 20, ib10
idx = 106
#%%
device = 'cuda'
fastgm = FastGM(uai_file=uai_file, device=device)
populate_gradient_factors(fastgm,iB=10)
bucket=fastgm.buckets[idx]
fastgm_copy = FastGM(uai_file=uai_file, device=device)
mg, mess = fastgm.get_message_gradient(idx)
gradient_factors = fastgm.get_gradient_factors(idx)
mg_hat = fastgm_copy.get_wmb_message_gradient(bucket_var=idx, i_bound=ib, weights='max')
mess.order_indices()
mg_hat.order_indices()

<class 'pyGMs.factor.Factor'>
<class 'pyGMs.factor.Factor'>


In [3]:
fastgm.config = {
    'num_samples': -1,
    'sampling_scheme': 'all'
}

In [4]:
sg = SampleGenerator(gm=fastgm, bucket=bucket)

In [5]:
sample_assignments = sg.sample_assignments(1000)
sample_values = sg.compute_message_values(sample_assignments)
sample_mg_values = sg.compute_gradient_values(sample_assignments, [mg])

In [6]:
# data_preprocessor = DataPreprocessor(y=None, mg=None, is_logspace=True, device = device)
data_preprocessor = DataPreprocessor(y=sample_values, mg=sample_mg_values, is_logspace=True, device = device)

In [7]:
dl = DataLoader(bucket=bucket, sample_generator=sg, data_preprocessor=data_preprocessor)

In [8]:
batches=dl.load_batches(10,5,mgh_factors=[mg])

In [9]:
y, mgh = data_preprocessor.convert_data()
x = data_preprocessor.one_hot_encode(bucket, sample_assignments, lower_dim=True)

In [10]:
# make the function linear to test learning
# y = x @ torch.tensor(np.arange(10) - 4.5).float().to(x.device)

In [11]:
# combine y and mghat for testing
# y = y + mgh

In [12]:
# batches[0]

In [ ]:
net = Net(input_size=len(batches[0]['x'][0]), hidden_sizes=[100,100], use_gradient_values=True, device=device)

In [29]:
device='cuda'
config = {
    'loss_fn': 'from_logspace_gil1c',
    'optimizer': 'adam',
    'learning_rate': 0.0000001,
    'device': device,
    'num_samples': 102400000,
    'batch_size': 1024,
    'num_epochs': 1,
    'set_size': 1024000,
    
    
    
}

In [30]:
t=Trainer(net, dataloader=dl, config=config)
t.train([mg])

Set 1/100, Epoch 1/1, Loss: -17.864173889160156
Set 2/100, Epoch 1/1, Loss: -17.86559295654297
Set 3/100, Epoch 1/1, Loss: -17.868091583251953
Set 4/100, Epoch 1/1, Loss: -17.872154235839844
Set 5/100, Epoch 1/1, Loss: -17.87765121459961
Set 6/100, Epoch 1/1, Loss: -17.87726402282715
Set 7/100, Epoch 1/1, Loss: -17.878446578979492
Set 8/100, Epoch 1/1, Loss: -17.87786293029785
Set 9/100, Epoch 1/1, Loss: -17.879337310791016
Set 10/100, Epoch 1/1, Loss: -17.8792724609375
Set 11/100, Epoch 1/1, Loss: -17.879133224487305
Set 12/100, Epoch 1/1, Loss: -17.878812789916992
Set 13/100, Epoch 1/1, Loss: -17.877517700195312
Set 14/100, Epoch 1/1, Loss: -17.87850570678711
Set 15/100, Epoch 1/1, Loss: -17.879817962646484
Set 16/100, Epoch 1/1, Loss: -17.8796329498291
Set 17/100, Epoch 1/1, Loss: -17.87922477722168
Set 18/100, Epoch 1/1, Loss: -17.879322052001953
Set 19/100, Epoch 1/1, Loss: -17.87926483154297
Set 20/100, Epoch 1/1, Loss: -17.880456924438477
Set 21/100, Epoch 1/1, Loss: -17.8803501

In [16]:
# t = Trainer(net, dataloader=None, config=config)
# t.train_epoch(batches)

In [17]:

# do_grad = config['loss_fn'] not in ['mse', 'l1', 'logspace_mse', 'from_logspace_l1', 'from_logspace_mse', 'logspace_l1']
# do_batches = True
# batch_size = 1024
# indices = []

# # for i in range(30000):
# for i in range(config['num_epochs']):
#     if do_batches:
#         # shuffle the batches
#         if i%10 == 0:
#             indices = torch.randperm(len(x))
#             x_shuffled = x[indices]
#             y_shuffled = y[indices]
#         # y_shuffled = sample_values[indices]
#         if do_grad:
#             mgh_shuffled = mgh[indices]
#         if do_grad:
#             batch1 = x_shuffled[:512], y_shuffled[:512], mgh_shuffled[:512]
#             batch2 = x_shuffled[512:], y_shuffled[512:], mgh_shuffled[512:]
#         else:
#             batch1 = x_shuffled[:512], y_shuffled[:512]
#             batch2 = x_shuffled[512:], y_shuffled[512:]
#         batch1_loss = t.train_batch(*batch1)
#         batch2_loss = t.train_batch(*batch2)
#     else:
#         if do_grad:
#             full_batch = x,y,mgh
#         else:
#             full_batch = x, y
#         t.train_batch(*full_batch)
    
        
#     if i % 100 == 0:
#         with torch.no_grad():
#             print('epoch: ', i)
#             if do_grad:
#                 print('loss is ', t.loss_fn(net(x).squeeze(), y, mgh).item())
#             else:
#                 print('loss is ', t.loss_fn(net(x).squeeze(), y).item())
    
    


In [27]:
Z = (mess * mg).sum_all_entries()
print(Z)

291.7323913574219


In [31]:
# get partition function estimate
# Z_linear_test = 316.2219543457031
mess_hat = FastFactor.nn_to_FastFactor(idx, fastgm, data_preprocessor, net=net, device=device)
# mess_hat = FastFactor.nn_to_FastFactor(idx, fastgm, None, net=net, device=device)
(mess_hat*mg).sum_all_entries() - Z

-0.195556640625

In [16]:
net(x).squeeze()[553]

tensor(-0.0265, device='cuda:0', grad_fn=<SelectBackward0>)

In [17]:
y[553]

tensor(-0.0457, device='cuda:0')

In [37]:
mess.tensor.max()

tensor(250.0704, device='cuda:0')

-0.188446044921875

In [17]:
net(x).squeeze()[713]

tensor(0.4032, device='cuda:0', grad_fn=<SelectBackward0>)

In [26]:
torch.abs(net(x).squeeze() - y).max()

tensor(22.9636, device='cuda:0', grad_fn=<MaxBackward1>)

In [34]:
net(x).squeeze()[492]
y[492]

tensor(-0.0457, device='cuda:0')

In [35]:
torch.argmax((net(x).squeeze() - y) * mgh)

tensor(492, device='cuda:0')

In [36]:
((net(x).squeeze() - y) * mgh)[492]

tensor(0.0248, device='cuda:0', grad_fn=<SelectBackward0>)

304.7890930175781

In [5]:
tensor=torch.rand(2,3,4,6,7)
assignments = torch.IntTensor([[0,0,0,0,0],[1,2,3,4,5]])
labels = [1,2,3,5,6] # of factor
scope = [1,2,3,4,5] # of message
elim_vars = [6]
elim_indices = [i for i, idx in enumerate(labels) if idx in elim_vars]
print('pre permutation elim indices: ', elim_indices)
assignment_indices = [i for i, idx in enumerate(scope) if idx in labels]
tensor_assignment_indices = [i for i, idx in enumerate(labels) if idx not in elim_vars]
print('pre permutation tensor_assignment_indices: ', tensor_assignment_indices)
permutation = (*tensor_assignment_indices, *elim_indices)
print('permutation: ', permutation)
tensor = tensor.permute(permutation)
# reordered labels
labels = [labels[i] for i in permutation]
# for loop through the columns of the tensor, so yes, this is efficient
# assignment_indices = [i for i, idx in enumerate(labels) if idx in scope]
print('assignment_indices are ', assignment_indices)
projected_assignments = assignments[:,assignment_indices]
print('projected_assignments are', projected_assignments)

pre permutation elim indices:  [4]
pre permutation tensor_assignment_indices:  [0, 1, 2, 3]
permutation:  (0, 1, 2, 3, 4)
assignment_indices are  [0, 1, 2, 4]
projected_assignments are tensor([[0, 0, 0, 0],
        [1, 2, 3, 5]], dtype=torch.int32)


In [105]:
projected_assignments.T

tensor([[0, 1],
        [0, 2],
        [0, 3],
        [0, 5]], dtype=torch.int32)

In [112]:
tensor.shape
projected_assignments.T

tensor([[0, 1],
        [0, 2],
        [0, 3],
        [0, 5]], dtype=torch.int32)

In [107]:
tensor[:,[0,1],[0,2],[0,3],[0,5]]

tensor([[0.5187, 0.2667],
        [0.0743, 0.7428],
        [0.1504, 0.0098],
        [0.0135, 0.9487],
        [0.9570, 0.0108],
        [0.4947, 0.4307],
        [0.6047, 0.6806]])

In [110]:
t2 = torch.rand(2,3,4,6)
t2[projected_assignments.T]

IndexError: index 2 is out of bounds for dimension 0 with size 2

In [42]:
slices = [indices[:, dim].unsqueeze(-1) for dim in range(indices.shape[1])]

In [43]:
slices

[tensor([[0],
         [1]]),
 tensor([[0],
         [2]]),
 tensor([[0],
         [3]]),
 tensor([[0],
         [4]])]

In [25]:
import torch
a = torch.rand(2,3,4,5)
a.view(torch.tensor(a.shape[:2]), torch.prod(torch.tensor(a.shape[2:])))


TypeError: view() received an invalid combination of arguments - got (Tensor, Tensor), but expected one of:
 * (torch.dtype dtype)
 * (tuple of ints size)


In [39]:
tuple(int(dim) for dim in a.shape[:2])

(2, 3)

In [7]:
elim_vars = [6,7]
tensor=torch.rand(3,3,3,3)
tensor



tensor([[[[0.6675, 0.2818, 0.5232],
          [0.7850, 0.4291, 0.4799],
          [0.9067, 0.4759, 0.1384]],

         [[0.0836, 0.9033, 0.0685],
          [0.5516, 0.6661, 0.5801],
          [0.3347, 0.4222, 0.7964]],

         [[0.4056, 0.7190, 0.1852],
          [0.3702, 0.8057, 0.1589],
          [0.5209, 0.2540, 0.4539]]],


        [[[0.2109, 0.1926, 0.6154],
          [0.2832, 0.4957, 0.8275],
          [0.7963, 0.1209, 0.8235]],

         [[0.4514, 0.0938, 0.6211],
          [0.2247, 0.4675, 0.0014],
          [0.1073, 0.7820, 0.9860]],

         [[0.1778, 0.3909, 0.0477],
          [0.4185, 0.5575, 0.8668],
          [0.6914, 0.4952, 0.5116]]],


        [[[0.2281, 0.4498, 0.6238],
          [0.4719, 0.2846, 0.0985],
          [0.3959, 0.9456, 0.0308]],

         [[0.6465, 0.4795, 0.9703],
          [0.8482, 0.4505, 0.3884],
          [0.7556, 0.3110, 0.8642]],

         [[0.6489, 0.6739, 0.8286],
          [0.8991, 0.5311, 0.7211],
          [0.0215, 0.3400, 0.5150]]]])

In [12]:
tensor.reshape(3,3,3,1,3)


tensor([[[[[0.6675, 0.2818, 0.5232]],

          [[0.7850, 0.4291, 0.4799]],

          [[0.9067, 0.4759, 0.1384]]],


         [[[0.0836, 0.9033, 0.0685]],

          [[0.5516, 0.6661, 0.5801]],

          [[0.3347, 0.4222, 0.7964]]],


         [[[0.4056, 0.7190, 0.1852]],

          [[0.3702, 0.8057, 0.1589]],

          [[0.5209, 0.2540, 0.4539]]]],



        [[[[0.2109, 0.1926, 0.6154]],

          [[0.2832, 0.4957, 0.8275]],

          [[0.7963, 0.1209, 0.8235]]],


         [[[0.4514, 0.0938, 0.6211]],

          [[0.2247, 0.4675, 0.0014]],

          [[0.1073, 0.7820, 0.9860]]],


         [[[0.1778, 0.3909, 0.0477]],

          [[0.4185, 0.5575, 0.8668]],

          [[0.6914, 0.4952, 0.5116]]]],



        [[[[0.2281, 0.4498, 0.6238]],

          [[0.4719, 0.2846, 0.0985]],

          [[0.3959, 0.9456, 0.0308]]],


         [[[0.6465, 0.4795, 0.9703]],

          [[0.8482, 0.4505, 0.3884]],

          [[0.7556, 0.3110, 0.8642]]],


         [[[0.6489, 0.6739, 0.8286]],

     

In [43]:
view

(3, 3, 3)

In [10]:
b = a
b = b.permute(1,0)

In [11]:
b

tensor([[0.8012, 0.6901, 0.9071],
        [0.4665, 0.9279, 0.7756],
        [0.4795, 0.9028, 0.7312]])

In [16]:
a = torch.rand(5)
assert([a[i]<a[i+1] for i in range(3)])

In [20]:
assert(all([]))